In [1]:
# ─── Block 1: SETUP + TOP-100 USDT-M FUTURES (last 21 days) ────────────────
from datetime import datetime, timezone, timedelta
import ccxt, pandas as pd, time
from bisect import bisect_right

# 0) FIXED WINDOW: days ago → now
spans    = 9
now_utc  = datetime.now(timezone.utc)
start    = now_utc - timedelta(days=spans)
end      = now_utc

# print(f"→ Window: {start.isoformat()} → {end.isoformat()}")

# to ms, bump exact-midnight end forward by 1 ms
start_ms = int(start.timestamp() * 1000)
end_ms   = int(  end.timestamp() * 1000)
if end_ms % 86_400_000 == 0:
    end_ms += 86_400_000 - 1

# 1) CCXT FUTURES CLIENT
binance = ccxt.binance({
    'options':        {'defaultType': 'future'},
    'enableRateLimit': True,
})


# 2) TRY to GET 24 h STATS
try:
    n=100
    raw24 = binance.fapiPublicGetTicker24hr()    # GET /fapi/v1/ticker/24hr
    usdt  = [t for t in raw24 if t['symbol'].endswith('USDT')]
    topn  = sorted(usdt, key=lambda x: float(x['quoteVolume']), reverse=True)[:n]
    symbols = [ t['symbol'][:-4] + '/USDT' for t in topn ]
    # print(f"Top {len(symbols)} USDT-M futures by 24 h volume:")
    # print(symbols)
except Exception as e:
    # print(f"⚠️  Cannot fetch futures tickers ({e}), falling back to symbols.txt")
    # Load a pre-prepared file of one-per-line SYMBOL/USDT
    with open('symbols.txt') as f:
        symbols = [l.strip() for l in f if l.strip()]



In [2]:
# ─── Block 2: fetch_ohlcv_until() + scan_symbol() ───────────────────────────
import numpy as np
from ta.trend      import MACD, ADXIndicator, EMAIndicator
from ta.volatility import AverageTrueRange
from ta.momentum   import RSIIndicator
import ta  # for MFI

def fetch_ohlcv_until(symbol, timeframe, start_ms, end_ms, limit=1500):
    all_bars, cursor = [], start_ms
    delay = binance.rateLimit/1000
    while True:
        batch = binance.fetch_ohlcv(symbol, timeframe, since=cursor, limit=limit)
        if not batch: break
        last = batch[-1][0]
        all_bars += batch
        # print(f"  • fetched {len(batch)} bars → {pd.to_datetime(last,unit='ms')} UTC")
        if last >= end_ms:
            all_bars = [b for b in all_bars if b[0] <= end_ms]
            # print("    ▶ reach end_ms, stopping.")
            break
        cursor = last+1
        time.sleep(delay)
        if len(batch)<limit: break

    df = pd.DataFrame(all_bars, columns=['ts','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['ts'],unit='ms')
    df.set_index('timestamp',inplace=True)
    df.drop('ts',axis=1,inplace=True)
    df = df[(df.index>=pd.to_datetime(start_ms,unit='ms'))
          & (df.index<=pd.to_datetime(end_ms,  unit='ms'))]
    df.index = df.index.tz_localize('UTC').tz_convert('Asia/Bangkok')
    return df

def safe_round(value, ndigits=0):
    if isinstance(value, (int, float)):
        return round(value, ndigits)
    return value  # Leave it unchanged if not a number

def scan_symbol(symbol, start_ms, end_ms):
    print(f"▶ scanning {symbol}")
    # 1) fetch 15m + warmup 4h
    df15 = fetch_ohlcv_until(symbol, '15m', start_ms, end_ms)
    MS4H = 4*60*60*1000
    warm = start_ms - (14*10 + 5)*MS4H
    raw4 = fetch_ohlcv_until(symbol, '4h', warm, end_ms)
    # slice raw4 → exact window
    sdt = pd.to_datetime(start_ms,unit='ms',utc=True).tz_convert('Asia/Bangkok')
    edt = pd.to_datetime(end_ms,  unit='ms',utc=True).tz_convert('Asia/Bangkok')
    df4  = raw4.loc[sdt:edt]

    # 2) merge 4h → 15m
    # ─── 4h → 15m ATR & ADX (guard against too-short history + catch errors) ──────────────────
    WINDOW = 14

    if len(raw4) < WINDOW + 1:
        # Not enough 4h bars to compute a 14-period indicator
        atr4 = pd.Series(0.0, index=df15.index)
        adx4 = pd.Series(0.0, index=df15.index)
    else:
        # ATR(14)
        atr4 = AverageTrueRange(
            high= raw4['high'],
            low=  raw4['low'],
            close=raw4['close'],
            window=WINDOW
        ).average_true_range()

        # ADX(14) with exception handling
        try:
            adx_series = ADXIndicator(
                high= raw4['high'],
                low=  raw4['low'],
                close=raw4['close'],
                window=WINDOW,
                fillna=False
            ).adx()
            adx4 = adx_series.fillna(0)
        except Exception as e:
            # print(f"    ⚠️ ADX calculation failed for {symbol}: {e}. Zero-filling it.")
            adx4 = pd.Series(0.0, index=raw4.index)

    # forward-fill onto the 15m frame
    df15['ATR_4h'] = atr4.reindex(df15.index, method='ffill')
    df15['ADX_4h'] = adx4.reindex(df15.index, method='ffill')

    # 3) 15m indicators
    macd = MACD(df15['close'])
    df15['MACD_line']   = macd.macd()
    df15['MACD_signal'] = macd.macd_signal()
    df15['MACD_hist']   = macd.macd_diff()
    df15['MFI']   = ta.volume.money_flow_index(df15['high'],df15['low'],df15['close'],df15['volume'],14)
    df15['EMA50'] = EMAIndicator(df15['close'],50).ema_indicator()
    df15['EMA100']= EMAIndicator(df15['close'],100).ema_indicator()
    df15['RSI']   = RSIIndicator(df15['close'],14).rsi()
    df15['ATR']   = AverageTrueRange(df15['high'],df15['low'],df15['close'],14).average_true_range()
    df15['ATR30']= df15['ATR'].rolling(30).mean()
    df15['ADX']  = ADXIndicator(df15['high'],df15['low'],df15['close'],14).adx()
    df15.dropna(inplace=True)

    # 4) pivots
    pw = 14

    # 1) Trailing pivot highs: current high is the max of itself + previous pw bars
    df15['is_pivot_high'] = (
        df15['high']
        .rolling(window=pw+1)                # trailing window of size pw+1
        .apply(lambda x: np.argmax(x) == pw, raw=True)
    )

    # 2) Trailing pivot lows: current low is the min of itself + previous pw bars
    df15['is_pivot_low']  = (
        df15['low']
        .rolling(window=pw+1)
        .apply(lambda x: np.argmin(x) == pw, raw=True)
    )

    # 3) Extract the times & values for your R:R logic
    highs = df15.loc[df15['is_pivot_high'] == 1, 'high']
    lows  = df15.loc[df15['is_pivot_low']  == 1, 'low']

    HT, HV = highs.index.tolist(), highs.values.tolist()
    LT, LV = lows.index.tolist(),  lows.values.tolist()

    # 5) backtest exactly as before
    results, active = [], False
    lev, cap, base_rr, rr_min, th = 10, 25, 1.5, 0.15, 25
    risk_th = 0.75
    fee = 0.0005
    for i in range(1,len(df15)):
        r, p = df15.iloc[i], df15.iloc[i-1]
        now  = r.name

        if not active:
            # trend filter
            if r['ADX_4h']<th: continue
            else:
                # LONG
                if ( r['high']>=r['EMA50'] and r['EMA50']>r['EMA100']
                and p['MFI']<=55        and r['MFI']>=55
                and r['MACD_hist']>0):
                    entry, entry_t, side = r['close'], now, 'long'

                    risk = r['ATR_4h']*risk_th
                    stop = entry-risk
                    idx  = bisect_right(HT,entry_t)
                    cand = HV[idx] if idx<len(HV) else entry+base_rr*risk
                    tp   = cand if cand>entry else entry+base_rr*risk
                    rr   = (tp-entry)/risk
                
                    # ❌ only go active if R:R > RR_MIN
                    if rr <= rr_min:
                        continue

                    active = True

                # SHORT
                elif ( r['low']<=r['EMA50'] and r['EMA50']<r['EMA100']
                    and p['MFI']>=45 and r['MFI']<=45
                    and r['MACD_hist']<0):
                    entry, entry_t, side = r['close'], now, 'short'

                    risk = r['ATR_4h']*risk_th
                    stop = entry+risk
                    idx  = bisect_right(LT,entry_t)
                    cand = LV[idx] if idx<len(LV) else entry-risk*base_rr
                    tp   = cand if cand<entry else entry-risk*base_rr
                    rr   = abs((tp-entry)/risk)

                    # ❌ only go active if R:R > 1
                    if rr <= rr_min:
                        continue
                    
                    active = True

                else:
                    continue

        else:
            lo, hi = r['low'], r['high']
            if side=='long':
                if lo<=stop: result,exit_price='loss',stop
                elif hi>=tp:   result,exit_price='win', tp
                else:          continue
            else:
                if hi>=stop:   result,exit_price='loss',stop
                elif lo<=tp:   result,exit_price='win', tp
                else:          continue

            # pnl + fees
            # pnl_pct = ((exit_price-entry)/entry)*(1 if side=='long' else -1)*lev*100
            # pnl_usd = cap*pnl_pct/100
            # not_usd = cap*lev
            # qty     = safe_round(not_usd/entry,4)
            # fees    = qty*(entry+exit_price)*0.5*fee
            # net     = pnl_usd - fees

            results.append({
                'symbol':      symbol,
                'side':        side,
                'entry_time':  entry_t,
                'exit_time':   now,
                'entry_price': entry,
                'stop_loss':   stop,
                'take_profit': tp,
                # 'exit_price':  exit_price,
                # 'volume_$':    safe_round(not_usd,2),
                # 'fees_$':      safe_round(fees,4),
                # 'pnl_%':       safe_round(pnl_pct,2),
                # 'pnl_$':       safe_round(pnl_usd,2),
                # 'net_pnl_$':   safe_round(net,2),
                'R:R':         safe_round(rr,2)
            })
            active=False
        
        if active: 
            # pnl_pct = None
            # pnl_usd = None
            # not_usd = cap*lev
            # qty     = safe_round(not_usd/entry,4)
            # fees    = None
            # net     = None
        
            results.append({
                'symbol':      symbol,
                'side':        side,
                'entry_time':  entry_t,
                'exit_time':   None,
                'entry_price': entry,
                'stop_loss':   stop,
                'take_profit': tp,
                # 'exit_price':  None,
                # 'volume_$':    safe_round(not_usd,2),
                # 'fees_$':      safe_round(fees,4),
                # 'pnl_%':       safe_round(pnl_pct,2),
                # 'pnl_$':       safe_round(pnl_usd,2),
                # 'net_pnl_$':   safe_round(net,2),
                'R:R':         safe_round(rr,2)
            })
    
        
    # Build results DataFrame
    result_df = pd.DataFrame(results)

    # Keep only the final trade (or return an empty DataFrame if there were none)
    result_df = result_df.tail(1).reset_index(drop=True)

    return result_df


In [3]:
# ─── Block 3: scan each symbol in parallel and collect only still-open trades ──
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

# rate_limit_sec = binance.rateLimit / 5000  

def _process_symbol(sym):
    # time.sleep(rate_limit_sec)
    try:
        df_trades = scan_symbol(sym, start_ms, end_ms)
        if df_trades is None or df_trades.empty or 'exit_time' not in df_trades:
            return None
        still_open = df_trades[df_trades['exit_time'].isna()].copy()
        if still_open.empty:
            return None
        still_open['symbol'] = sym
        return still_open
    except Exception as e:
        print(f"⚠️ Error processing {sym}: {e}")
        return None

open_trades = []
max_workers = 5
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(_process_symbol, sym): sym for sym in symbols}
    for fut in as_completed(futures):
        res = fut.result()
        if res is not None:
            open_trades.append(res)

print(f"→ Window: {start.isoformat()} → {end.isoformat()}")



▶ scanning ETH/USDT
▶ scanning BTC/USDT
▶ scanning SOL/USDT
▶ scanning ALPACA/USDT
▶ scanning 1000PEPE/USDT


▶ scanning AXL/USDT


▶ scanning XRP/USDT
▶ scanning DOGE/USDT


▶ scanning RESOLV/USDT
▶ scanning UNI/USDT
▶ scanning AAVE/USDT


▶ scanning SUI/USDT
▶ scanning FARTCOIN/USDT


...

▶ scanning OM/USDT
▶ scanning OCEAN/USDT


▶ scanning DEXE/USDT
▶ scanning APT/USDT
▶ scanning ORDI/USDT


▶ scanning GALA/USDT
▶ scanning HBAR/USDT


▶ scanning EPT/USDT
▶ scanning STRAX/USDT
▶ scanning SSV/USDT


→ Window: 2025-06-02T04:45:07.543842+00:00 → 2025-06-11T04:45:07.543842+00:00


In [4]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

if open_trades:
    df_active = pd.concat(open_trades, ignore_index=True)

    # Clean up DataFrame
    df_active = df_active.drop(columns=["exit_time"], errors="ignore")
    df_active = df_active.sort_values("entry_time", ascending=False)
    df_active["entry_time"] = pd.to_datetime(df_active["entry_time"]).dt.strftime("%Y-%m-%d %H:%M")
    
    # Round float columns for readability
    float_cols = df_active.select_dtypes(include='number').columns
    df_active[float_cols] = df_active[float_cols].round(4)

    # Prepare formatted string
    formatted = df_active.tail(10).to_string(index=False)

    with open("summary.txt", "w") as f:
        f.write(f"{timestamp}\n")
        f.write(formatted + "\n")

    # Display in notebook
    print("\n📋 Active orders:")
    display(df_active)

else:
    with open("summary.txt", "w") as f:
        f.write(f"{timestamp}\n")
        f.write("No active trades at the moment.\n")



📋 Active orders:


,symbol,side,entry_time,entry_price,stop_loss,take_profit,R:R
3,HYPE/USDT,long,2025-06-11 11:45,41.4970,40.4838,43.0168,1.5
2,COMP/USDT,long,2025-06-11 11:30,61.3900,59.1805,64.7042,1.5
8,FIL/USDT,long,2025-06-11 11:30,2.7130,2.6734,2.7725,1.5
9,UMA/USDT,short,2025-06-11 11:30,1.5210,1.5822,1.4292,1.5
10,1000BONK/USDT,long,2025-06-11 11:30,0.0177,0.0172,0.0183,1.5
0,SUI/USDT,long,2025-06-11 11:00,3.4700,3.4130,3.5555,1.5
5,VIRTUAL/USDT,long,2025-06-11 10:30,2.1348,2.0722,2.2287,1.5
7,1000000BOB/USDT,short,2025-06-11 10:00,0.0578,0.0651,0.0469,1.5
4,BCH/USDT,long,2025-06-11 08:15,442.3900,436.3402,451.4648,1.5
1,WIF/USDT,long,2025-06-11 06:45,1.0268,0.9925,1.0782,1.5
